In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
import pickle
from sklearn.ensemble import GradientBoostingClassifier
from skimage.feature import hog
from sklearn.model_selection import train_test_split
import glob2
%matplotlib inline

In [2]:
cars = glob2.glob('vehicles\**\*.png')
notcars = glob2.glob('non-vehicles\**\*.png')

In [48]:
### TODO: Tweak these parameters and see how the results change.
color_space = 'YUV' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_size = (14, 14) # Spatial binning dimensions
hist_bins = 16    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
y_start_stop = [None, None] # Min and max in y to search in slide_window()

In [4]:
# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

In [9]:
alldata = {}

(64, 64, 3)

In [85]:
alldata = {}

## HOG Features
features = []
for file in cars:
    image = mpimg.imread(file)
    feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
    
    hog_features = []
    for channel in range(feature_image.shape[2]):
        hog_features.append(get_hog_features(feature_image[:,:,channel], 
                            orient, pix_per_cell, cell_per_block, 
                            vis=False, feature_vec=True))
    hog_features = np.ravel(hog_features)   
    features.append(hog_features)
    
    
    

    
for file in notcars:
    image = mpimg.imread(file)
    feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
    
    hog_features = []
    for channel in range(feature_image.shape[2]):
        hog_features.append(get_hog_features(feature_image[:,:,channel], 
                            orient, pix_per_cell, cell_per_block, 
                            vis=False, feature_vec=True))
    hog_features = np.ravel(hog_features)   
    features.append(hog_features)
    
alldata["hogfeatures"] = features

In [86]:
y = np.hstack((np.ones(len(cars)), np.zeros(len(notcars))))
alldata["label"] = y

In [87]:
for each in alldata.keys():
    alldata[each] = np.array(alldata[each])

In [ ]:
pickle.dump( alldata, open( "Data.p", "wb" ) )

In [88]:
features = np.array(features)
features.shape

(17760, 5292)

In [89]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_idx, test_idx in sss.split(features, y):
    
    X_train, X_test = features[train_idx], features[test_idx]
    
    y_train, y_test = y[train_idx], y[test_idx]

In [5]:
data = pickle.load( open( "Data.p", "rb" ) )

In [59]:
modeldata = {}

modeldata["binpca"] = binpca

modeldata["hogpca"] = hogpca

pickle.dump( modeldata, open( "models.p", "wb" ) )

In [14]:
import sys
sys.path.append("C:/xgboost/python-package")

import xgboost as xgb

In [90]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [27]:
dtrain = xgb.DMatrix(features, label=y)


In [91]:
param = {'max_depth':2, 'eta':1, 'silent':1, 'objective':'binary:logistic' }
watchlist  = [(dtrain,'train'), (dtest,'eval')]
num_round = 155
bst = xgb.train(param, dtrain, num_round, watchlist, early_stopping_rounds=10)
#bst = xgb.train(param, dtrain, num_round)

[0]	train-error:0.179195	eval-error:0.185248
Multiple eval metrics have been passed: 'eval-error' will be used for early stopping.

Will train until eval-error hasn't improved in 10 rounds.
[1]	train-error:0.141258	eval-error:0.155124
[2]	train-error:0.111486	eval-error:0.118806
[3]	train-error:0.096073	eval-error:0.104167
[4]	train-error:0.083756	eval-error:0.08643
[5]	train-error:0.07714	eval-error:0.080236
[6]	train-error:0.068623	eval-error:0.073761
[7]	train-error:0.063274	eval-error:0.072635
[8]	train-error:0.057432	eval-error:0.060248
[9]	train-error:0.051872	eval-error:0.058559
[10]	train-error:0.049831	eval-error:0.061092
[11]	train-error:0.046382	eval-error:0.055743
[12]	train-error:0.042722	eval-error:0.052928
[13]	train-error:0.039062	eval-error:0.046171
[14]	train-error:0.036599	eval-error:0.043637
[15]	train-error:0.033713	eval-error:0.041667
[16]	train-error:0.031602	eval-error:0.040259
[17]	train-error:0.028998	eval-error:0.040822
[18]	train-error:0.026745	eval-error:0.

In [92]:
pred = bst.predict(dtest)
predictions = (pred > 0.5) * 1
predictions

array([0, 0, 0, ..., 0, 1, 1])

In [93]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.97409909909909909

In [94]:
bst.save_model("xgbcrb.model")

In [74]:
fscore = bst.get_fscore()

In [75]:
fscore

{'f1': 1,
 'f1008': 1,
 'f1016': 2,
 'f1048': 1,
 'f1062': 3,
 'f1074': 1,
 'f1075': 1,
 'f1114': 1,
 'f1138': 1,
 'f1146': 1,
 'f1148': 1,
 'f1166': 1,
 'f1174': 1,
 'f1193': 1,
 'f121': 2,
 'f1210': 1,
 'f1230': 1,
 'f1232': 1,
 'f1233': 1,
 'f1234': 1,
 'f1241': 3,
 'f1255': 1,
 'f1260': 1,
 'f1268': 1,
 'f1290': 1,
 'f1305': 1,
 'f132': 1,
 'f1363': 1,
 'f1372': 3,
 'f1390': 1,
 'f1395': 1,
 'f1399': 2,
 'f1400': 1,
 'f1417': 1,
 'f1421': 1,
 'f1426': 1,
 'f1431': 1,
 'f1436': 1,
 'f1455': 1,
 'f146': 1,
 'f1461': 1,
 'f1476': 1,
 'f1484': 1,
 'f1485': 1,
 'f1502': 1,
 'f1507': 1,
 'f1511': 2,
 'f1512': 2,
 'f1527': 1,
 'f1533': 2,
 'f1539': 1,
 'f1542': 1,
 'f1543': 1,
 'f1552': 1,
 'f1561': 1,
 'f1575': 1,
 'f1606': 1,
 'f1608': 1,
 'f1611': 1,
 'f1623': 1,
 'f1624': 2,
 'f1627': 1,
 'f1642': 2,
 'f1647': 1,
 'f1648': 1,
 'f165': 1,
 'f1651': 2,
 'f1660': 1,
 'f1661': 1,
 'f1668': 1,
 'f1677': 1,
 'f168': 1,
 'f1690': 1,
 'f1710': 1,
 'f1714': 1,
 'f1724': 1,
 'f1736': 1,
 'f1760

In [19]:

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))

Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 6108


In [ ]:
# Get the estimator
best_clf = grid_fit.best_estimator_

In [ ]:
image.shape